Данная лабораторная работа ориентирована на знакомство с деревьями решений с реализацией на языке программирования Python:

Реализовать решение задачи классификации пользовательских данных с использованием деревьев решений. Сравнить с ранее применявшимися методами.
Реализовать решение задачи регрессии для пользовательских данных с использованием деревьев решений. Сравнить с ранее применявшимися методами.
Реализовать решение задачи заполнения пропусков с использованием деревьев решений. Сравнить с ранее применявшимися методами.
Самостоятельно программно реализовать по крайней мере один из алгоритмов построения деревьев решений (ID3, C4.5, C5.0, CART).

In [1]:
from sklearn.datasets import load_breast_cancer
X, y=load_breast_cancer(return_X_y=True, as_frame=True)

In [2]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
sum(y==1)/len(y) #Процент людей с раком груди. Датасет +- сбалансированный

0.6274165202108963

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
param_dist = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 45,  50, 55, 60],
    'min_samples_split': [1, 2, 3, 4,  5, 10, 15, 20, 25, 30 , 40],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7,8, 10, 15, 20, 25, 30, 35, 40],
    'max_features': ['auto', 'sqrt', 'log2'],
    'class_weight': ['balanced', None],
    #'min_impurity_decrease' : uniform(0, 0.2),
    #'ccp_alpha' : uniform(0, 0.2)
}
dtree = DecisionTreeClassifier()
random_search = RandomizedSearchCV(estimator=dtree, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)
print("Лучшие параметры:", random_search.best_params_)
print("Лучшее качество модели:", random_search.best_score_)

Лучшие параметры: {'splitter': 'best', 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': 'auto', 'max_depth': 40, 'criterion': 'entropy', 'class_weight': None}
Лучшее качество модели: 0.9384615384615385


C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
45 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 250, in fit
    raise ValueError(
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  warnings.warn(som

In [6]:
from sklearn.metrics import accuracy_score
model=random_search.best_estimator_
model.fit(X_train, y_train)
print(accuracy_score(model.predict(X_test), y_test))

0.9649122807017544


In [10]:
from sklearn.svm import SVC


svm_model = SVC()
param_dist = {
    'C': [0.1, 1, 10, 100],
    #'C': uniform(loc=0, scale=40),
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']  
}

rand_search = RandomizedSearchCV(estimator=svm_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=10, random_state=42)
rand_search.fit(X_train, y_train)
print("Лучшие параметры:", rand_search.best_params_)
print("Лучшее качество модели:", rand_search.best_score_)

Лучшие параметры: {'kernel': 'linear', 'gamma': 'auto', 'C': 100}
Лучшее качество модели: 0.956043956043956


In [11]:
model=rand_search.best_estimator_
model.fit(X_train, y_train)
print(accuracy_score(model.predict(X_test), y_test))

0.9649122807017544


In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

knn_pipe = Pipeline([
    ('scaler', StandardScaler()),      
    ('knn', KNeighborsClassifier())       
])

param_dist = {
    'knn__n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25],       
    'knn__weights': ['uniform', 'distance'],                    
}

rand_search = RandomizedSearchCV(estimator=knn_pipe, param_distributions=param_dist, cv=5, n_iter=30, scoring='accuracy',  n_jobs=10)
rand_search.fit(X_train, y_train)

print("Лучшие параметры:", rand_search.best_params_)
print("Лучшее качество модели:", rand_search.best_score_)

C:\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 24 is smaller than n_iter=30. Running 24 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Лучшие параметры: {'knn__weights': 'distance', 'knn__n_neighbors': 8}
Лучшее качество модели: 0.9736263736263737


In [14]:
model=rand_search.best_estimator_
model.fit(X_train, y_train)
print(accuracy_score(model.predict(X_test), y_test))

0.9736842105263158


In [1]:
from sklearn.datasets import fetch_california_housing
X, y=fetch_california_housing(return_X_y=True, as_frame=True)
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'criterion' : ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 45,  50, 55, 60],
    'min_samples_split': [1, 2, 3, 4,  5, 10, 15, 20, 25, 30 , 40],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7,8, 10, 15, 20, 25, 30, 35, 40],
    'max_features': ['auto', 'sqrt', 'log2'],
    #'min_impurity_decrease' : uniform(0, 0.2),
    #'ccp_alpha' : uniform(0, 0.2)
}
dtree = DecisionTreeRegressor()
random_search = RandomizedSearchCV(estimator=dtree, param_distributions=param_dist,  n_iter=40, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)
print("Лучшие параметры:", random_search.best_params_)
print("Лучшее качество модели:", random_search.best_score_)

Лучшие параметры: {'splitter': 'best', 'min_samples_split': 25, 'min_samples_leaf': 6, 'max_features': 'auto', 'max_depth': 20, 'criterion': 'friedman_mse'}
Лучшее качество модели: -0.37555495939056344


C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 1315, in fit
    super().fit(
  File "C:\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 250, in fit
    raise ValueError(
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  warnings.warn(so

In [4]:
from sklearn.metrics import mean_squared_error
model=random_search.best_estimator_
model.fit(X_train, y_train)
print(mean_squared_error(model.predict(X_test), y_test))

0.36688586754511826


In [5]:
from sklearn.svm import SVR

svm_model = SVR(C=1, kernel='rbf', gamma='auto')
svm_model.fit(X_train, y_train)

SVR(C=1, gamma='auto')

In [6]:

print(mean_squared_error(svm_model.predict(X_test), y_test))

1.1671454913141537


In [9]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
knn_pipe = Pipeline([
    ('scaler', StandardScaler()),      
    ('knn', KNeighborsRegressor())       
])

param_dist = {
    'knn__n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25],       
    'knn__weights': ['uniform', 'distance'],                    
}

rand_search = RandomizedSearchCV(estimator=knn_pipe, param_distributions=param_dist, cv=5, n_iter=30, scoring='neg_mean_squared_error',  n_jobs=10)
rand_search.fit(X_train, y_train)

print("Лучшие параметры:", rand_search.best_params_)
print("Лучшее качество модели:", rand_search.best_score_)

C:\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 24 is smaller than n_iter=30. Running 24 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Лучшие параметры: {'knn__weights': 'distance', 'knn__n_neighbors': 9}
Лучшее качество модели: -0.4047386594377632


In [10]:
model=rand_search.best_estimator_
model.fit(X_train, y_train)
print(mean_squared_error(model.predict(X_test), y_test))

0.41914274251715045


In [12]:
from sklearn.datasets import load_diabetes
import numpy as np
X, y=load_diabetes(return_X_y=True, as_frame=True)
indices_to_remove = np.random.choice(X.index, 80, replace=False)
removed_age_values = X.loc[indices_to_remove, 'age']
X.loc[indices_to_remove, 'age'] = None
print(X, y)

          age       sex       bmi        bp        s1        s2        s3  \
0    0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1   -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2    0.085299  0.050680  0.044451 -0.005671 -0.045599 -0.034194 -0.032356   
3   -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4         NaN -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   
..        ...       ...       ...       ...       ...       ...       ...   
437  0.041708  0.050680  0.019662  0.059744 -0.005697 -0.002566 -0.028674   
438 -0.005515  0.050680 -0.015906 -0.067642  0.049341  0.079165 -0.028674   
439  0.041708  0.050680 -0.015906  0.017282 -0.037344 -0.013840 -0.024993   
440 -0.045472 -0.044642  0.039062  0.001215  0.016318  0.015283 -0.028674   
441       NaN -0.044642 -0.073030 -0.081414  0.083740  0.027809  0.173816   

           s4        s5        s6  
0   -0.002592  0.019908 -0.017646  
1  

In [14]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
data_mean = X.copy()
data_knn = X.copy()


mean_age = data_mean['age'].mean()
data_mean['age'].fillna(mean_age, inplace=True)



imputer = KNNImputer(n_neighbors=5)
data_knn['age'] = pd.DataFrame(imputer.fit_transform(data_knn))[0]  #
#print(data_knn['age'])

true_age = removed_age_values



filled_mean = data_mean.loc[indices_to_remove, 'age']
filled_knn = data_knn.loc[indices_to_remove, 'age']


mse_mean = mean_squared_error(true_age, filled_mean)
rmse_mean = math.sqrt(mse_mean)
mae_mean = mean_absolute_error(true_age, filled_mean)
mape_mean = np.mean(np.abs((true_age - filled_mean) / true_age)) 
smape_mean =  np.mean(2 * np.abs(filled_mean - true_age) / (np.abs(filled_mean) + np.abs(true_age)))
wape_mean = np.sum(np.abs(filled_mean - true_age)) / np.sum(np.abs(true_age))


mse_knn = mean_squared_error(true_age, filled_knn)
rmse_knn = math.sqrt(mse_knn)
mae_knn = mean_absolute_error(true_age, filled_knn)
mape_knn = np.mean(np.abs((true_age - filled_knn) / true_age)) 
smape_knn = np.mean(2 * np.abs(filled_knn - true_age) / (np.abs(filled_knn) + np.abs(true_age)))
wape_knn = np.sum(np.abs(filled_knn - true_age)) / np.sum(np.abs(true_age))


print(f"Mean Imputation - MSE: {mse_mean}, RMSE: {rmse_mean}, MAE: {mae_mean}, MAPE: {mape_mean}, SMAPE: {smape_mean}, WAPE: {wape_mean}")
print(f"KNN Imputation - MSE: {mse_knn}, RMSE: {rmse_knn}, MAE: {mae_knn}, MAPE: {mape_knn}, SMAPE: {smape_knn}, WAPE: {wape_knn}")

Mean Imputation - MSE: 0.001998376311698617, RMSE: 0.04470320247698835, MAE: 0.03674784271126715, MAPE: 1.01036488326491, SMAPE: 1.8675915528056457, WAPE: 1.0069595467635004
KNN Imputation - MSE: 0.0020224095011194417, RMSE: 0.04497120746788373, MAE: 0.03534459912842027, MAPE: 2.4365829979431815, SMAPE: 1.2489841144426883, WAPE: 0.9685080508951717


In [21]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error

df=X
known_age = df[df['age'].notna()]
unknown_age = df[df['age'].isna()]
y_train=known_age['age']
X_train=known_age.drop(['age'], axis=1)

y_test=unknown_age['age']
X_test=unknown_age.drop(['age'], axis=1)





param_dist = {
    'criterion' : ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 45,  50, 55, 60],
    'min_samples_split': [1, 2, 3, 4,  5, 10, 15, 20, 25, 30 , 40],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7,8, 10, 15, 20, 25, 30, 35, 40],
    'max_features': ['auto', 'sqrt', 'log2'],
    #'min_impurity_decrease' : uniform(0, 0.2),
    #'ccp_alpha' : uniform(0, 0.2)
}
dtree = DecisionTreeRegressor()
random_search = RandomizedSearchCV(estimator=dtree, param_distributions=param_dist,  n_iter=40, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)
print("Лучшие параметры:", random_search.best_params_)
print("Лучшее качество модели:", random_search.best_score_)


print(mean_squared_error(removed_age_values, random_search.predict(X_test)))

Лучшие параметры: {'splitter': 'best', 'min_samples_split': 4, 'min_samples_leaf': 10, 'max_features': 'auto', 'max_depth': 40, 'criterion': 'absolute_error'}
Лучшее качество модели: -2.3944663686137243e-05
0.0040135262638679005


C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
35 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 1315, in fit
    super().fit(
  File "C:\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 178, in fit
    raise ValueError(
ValueError: Some value(s) of y are negative which is not allowed for Poisson regression.

-------------------------------------

In [37]:
!pip install category_encoders

Defaulting to user installation because normal site-packages is not writeable


In [38]:
!pip install deepdiff

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [39]:
import pandas as pd
import numpy as np
from array import array
from copy import deepcopy
from deepdiff import DeepDiff
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator
from category_encoders import *
from typing import List, Union, Tuple
import pprint
def compute_impurity (target_vector: np.array, criterion: str = 'gini') -> float:
    unique_vals, probas=np.unique(target_vector, return_counts=True)
    probas=probas/np.sum(probas)
    if criterion=='gini' :
        return probas@(1-probas)
    else :
        return -np.sum([p*np.log2(p) for p in probas if p>0])

def compute_criterion(target_vector: np.array, feature_vector: np.array, threshold: float, criterion: str = 'gini') -> float:
    assert criterion in ['gini', 'entropy'], "Критерий может быть только 'gini' или 'entropy'!"

    pass
    if  len(np.unique(feature_vector))==1 :
        return 0
    n=len(target_vector)
    n_left=sum(feature_vector<=threshold)
    n_right=sum(feature_vector>threshold)
    H_R=compute_impurity(target_vector=target_vector, criterion=criterion)
    H_Rl=compute_impurity(target_vector=target_vector[feature_vector<=threshold], criterion=criterion)
    H_Rr=compute_impurity(target_vector=target_vector[feature_vector>threshold], criterion=criterion)
    Q=H_R-n_left/n*H_Rl-n_right/n*H_Rr
    return Q

def find_best_split(feature_vector: np.ndarray, target_vector: np.ndarray, criterion: str = 'gini') -> Tuple:
    unq_vals = np.sort(np.unique(feature_vector))

    if len(unq_vals) == 1:
        return None, None, None, 0

    pass
    thresholds=(unq_vals[1:]+unq_vals[:-1])/2
    criterion_vals=np.array([compute_criterion(target_vector=target_vector, feature_vector=feature_vector, threshold=t, criterion=criterion) for t in thresholds])
    threshold_best=thresholds[0]
    criterion_best=criterion_vals[0]
    for t in range(len(criterion_vals)) :
        if criterion_vals[t]>criterion_best :
            criterion_best=criterion_vals[t]
            threshold_best=thresholds[t]
    return thresholds, criterion_vals, threshold_best, criterion_best


class DecisionTree(BaseEstimator):

    def __init__(
            self,
            feature_types: list,
            criterion: str = 'gini',
            max_depth: int = None,
            min_samples_split: int = None,
    ):

        self._feature_types = feature_types
        self._tree = {}
        # Сюда будут сохраняться обученные таргет энкодеры категориальных фичей
        self.target_encodings = {} # Dict[int<номер категориальной фичи>, category_encoders.target_encoder.TargetEncoder]
        self._criterion = criterion
        self.max_depth = max_depth
        self._min_samples_split = min_samples_split
        self.cols=[]
        self.enc=1

    def _fit_node(self, sub_X: np.ndarray, sub_y: np.ndarray, node: dict, level: int):
        level+=1

        if level==self.max_depth or len(sub_X)<self._min_samples_split or len(np.unique(sub_y))==1 :
            
            dic={}
            unique_vals=self.classes
            probas=[0 for i in range(len(unique_vals))]
            for k in range(len(unique_vals)) :
                for r in range(len(sub_y)) :
                    if sub_y[r]==unique_vals[k] :
                        probas[k]+=1
            dic={'type': 'terminal', 'classes_distribution': np.array(probas)}
            #dic.update({'type': 'terminal'})
            #dic.update({'classes_distribution': probas}) 
        else :
            
            dic={}
            indicator=0
            for i in range(len(self._feature_types)) :
                feature_vector=sub_X[:, i]
                if len(np.unique(feature_vector))!=1 :
                    ans=find_best_split(feature_vector=feature_vector, target_vector=sub_y, criterion=self._criterion)
                    thresholds=ans[0]
                    criterion_vals=ans[1]
                    for j in range(len(thresholds)) :
                        if indicator==0 :
                            threshold_best= thresholds[j]
                            criterion_best=criterion_vals[j]
                            feature_best=i
                            indicator=1 
                        else :
                            if criterion_vals[j]>criterion_best :
                                threshold_best= thresholds[j]
                                criterion_best=criterion_vals[j]
                                feature_best=i
            
            feature_vector=sub_X[:, feature_best]
            if self._feature_types[feature_best]=='categorical' :
                
                dic={'type': 'nonterminal', 
                     'feature_type':  'categorical', 
                     'feature_number': feature_best, 
                     'threshold' : threshold_best,
                     'left_child': self._fit_node(sub_X=sub_X[feature_vector<=threshold_best], sub_y=sub_y[feature_vector<=threshold_best], node=self._tree, level=level),
                     'right_child': self._fit_node(sub_X=sub_X[feature_vector>threshold_best], sub_y=sub_y[feature_vector>threshold_best], node=self._tree, level=level)
                     }
                
            else :
                
                dic={'type': 'nonterminal', 
                     'feature_type':  'real', 
                     'feature_number': feature_best, 
                     'threshold' : threshold_best,
                     'left_child':  self._fit_node(sub_X=sub_X[feature_vector<=threshold_best], sub_y=sub_y[feature_vector<=threshold_best], node=self._tree, level=level),
                     'right_child': self._fit_node(sub_X=sub_X[feature_vector>threshold_best], sub_y=sub_y[feature_vector>threshold_best], node=self._tree, level=level)
                     }
            #dic.update({'feature_number': feature_best})
            #dic.update({'threshold' : threshold_best })
           # feature_vector=sub_X[:, feature_best]
           # dic.update({'left_child': self._fit_node(sub_X=sub_X[feature_vector<=threshold_best], sub_y=sub_y[feature_vector<=threshold_best], node=self._tree, level=level)})
           # dic.update({'right_child': self._fit_node(sub_X=sub_X[feature_vector>threshold_best], sub_y=sub_y[feature_vector>threshold_best], node=self._tree, level=level)})
        return dic

    def _predict_proba_object(self, x: np.array, node: dict) -> Union[List, np.ndarray]:

        #pprint.pprint(node['right_child'])
        if  node['type']=='terminal' :
            ans=node['classes_distribution']/np.sum(node['classes_distribution'])
            return ans
        else :
            if node['threshold']>x[node['feature_number']] :
                return self._predict_proba_object(x=x, node=node['left_child'])
            else :
                return self._predict_proba_object(x=x, node=node['right_child'])
    def fit(self, X: np.ndarray, y: np.ndarray):

        assert len(set(y)) > 1, 'Таргет должен содержать более одного класса!'
        
        X=pd.DataFrame(X)
        y=pd.DataFrame(y)
        
        
        for i in range(len(self._feature_types)) :
            if self._feature_types[i]=="categorical" :
                self.cols.append(i)
        #print(cols)
        if len(self.cols)!=0 :
            self.enc=TargetEncoder(cols=self.cols).fit(X, y)
            #print(self.enc)
            #print(self.enc.mapping)
            X=self.enc.transform(X)
        X=X.to_numpy()
        y=y.to_numpy()
        self.classes=np.unique(y)
        self.classes=np.sort(self.classes)
        if self._min_samples_split==None :
            self._min_samples_split=2
        self._tree =self._fit_node(sub_X=X, sub_y=y, node=self._tree, level=0)


    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        assert self._tree != {}, "Cначала обучите модель!"
        if len(self.cols)!=0 :
            X=pd.DataFrame(X)
            X=self.enc.transform(X)
            X=X.to_numpy()
       
        predicted = []
        for x in X:
            a=self._predict_proba_object(x, self._tree)
            predicted.append(a)
        return np.array(predicted)


    def predict(self, X: np.ndarray) -> np.ndarray:
        return np.argmax(self.predict_proba(X=X), axis=1).ravel()



In [29]:
from sklearn.datasets import load_breast_cancer
X, y=load_breast_cancer(return_X_y=True, as_frame=True)
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [42]:
max_depth=[1, 2,3 ,4 ,6, 7]
min_samples_split=[10, 15]
criterion=['gini', 'entropy']
feature_types=['real']*30
for x in criterion :
    for depth in max_depth:
        for split in  min_samples_split :
            dtree = DecisionTree(feature_types=feature_types, criterion=x, max_depth=depth, min_samples_split=split)
            dtree.fit(np.array(X_train), np.array(y_train))
            print(accuracy_score(dtree.predict(np.array(X_test)), y_test), ' criterion= ', x, ' max_depth= ', depth, ' min_samples_split=', split)


0.631578947368421  criterion=  gini  max_depth=  1  min_samples_split= 10
0.631578947368421  criterion=  gini  max_depth=  1  min_samples_split= 15
0.9210526315789473  criterion=  gini  max_depth=  2  min_samples_split= 10
0.9210526315789473  criterion=  gini  max_depth=  2  min_samples_split= 15
0.8947368421052632  criterion=  gini  max_depth=  3  min_samples_split= 10
0.8947368421052632  criterion=  gini  max_depth=  3  min_samples_split= 15
0.9385964912280702  criterion=  gini  max_depth=  4  min_samples_split= 10
0.9385964912280702  criterion=  gini  max_depth=  4  min_samples_split= 15
0.9122807017543859  criterion=  gini  max_depth=  6  min_samples_split= 10
0.9385964912280702  criterion=  gini  max_depth=  6  min_samples_split= 15
0.9122807017543859  criterion=  gini  max_depth=  7  min_samples_split= 10
0.9385964912280702  criterion=  gini  max_depth=  7  min_samples_split= 15
0.631578947368421  criterion=  entropy  max_depth=  1  min_samples_split= 10
0.631578947368421  criter